# PyQUBO Benchmark

En este cuaderno vamos a ver cómo afecta el tamaño y la conectividad de un problema QUBO a su resolución mediante la librería PyQUBO.

## QUBO

Forma general del QUBO (Quadratic Unconstrained Binary Optimization):

$$\min H (\vec{x}),\quad : \quad H=\sum_{i\geq j}^{N,N} h_{ij} x_ix_j\quad : \quad x_i,x_j\in\{0,1\} \quad \wedge\quad  h_{ij}\in\mathbb{R}$$

Esto recoge también términos lineales y constantes, porque $x_i\in\{0,1\} \Rightarrow x_i = x_i^2$ y al ser un problema de optimización los términos constantes no afectan a la resolución (son equivalentes a mover la "cota cero" de referencia de un paisaje, no desplaza el punto más elevado).

Los problemas QUBO son relevantes porque los ordenadores cuánticos son capaces de resolverlos:

    •	Los de recocido ("annealing") como los de DWave, de manera nativa
    •	Los de puertas lógicas, como los IBM, mediante el método QAOA (Quantum Approximated Optimization Algorithm)

### Hamiltoniano de restricción

Se puede añadir términos al hamiltoniano de la forma 

$$H^R= \lambda \left(x_i+x_j-0.5\right)^2 \quad : \lambda \in \mathbb{R}.$$

Esto hace que la variable $x_i$ y la variable $x_j$ no puedan activarse a la vez, puesto que:

$$\lambda\left(0+0-0.5\right)^2 = +0.25\lambda$$
$$\lambda\left(0+1-0.5\right)^2 = +0.25\lambda$$
$$\lambda\left(1+0-0.5\right)^2 = +0.25\lambda$$
$$\lambda\left(1+1-0.5\right)^2 = +2.25\lambda$$

y como este es un problema de minimización, si el $\lambda$ es lo suficientemente grande, sólo los tres primeros estados estarán permitidos.




## PyQUBO

Librería para resolución de QUBO de manera clásica aproximada.

•	https://pyqubo.readthedocs.io/en/latest/getting_started.html

•	https://github.com/recruit-communications/pyqubo/blob/master/docs/getting_started.rst

•	https://ar5iv.labs.arxiv.org/html/2103.01708

•	PyQUBO model can output the BinaryQuadraticModel(BQM) (Integration with DWave Ocean)

•	If you want to solve the problem by actual D-Wave machines, just replace the sampler by a DWaveCliqueSampler instance, for example.

## Comparativa

### QUBO a comparar:

$$H=-\sum_i^N x_i + \lambda_{adj}\left[\sum_i^{N-1}\left(x_i + x_{i+1} -\frac{1}{2} \right)^2+ \left(x_N + x_0 -\frac{1}{2}  \right)^2\right]+\lambda_{cross}\left[\sum_{(i, j)\in T}^{M}\left(x_i + x_j -\frac{1}{2}  \right)^2\right]\quad : \forall (i,j) \in T, j \neq \{i-1, \;i\;, i+1\}$$

Esto es: "toma todos los elementos que puedas, sin tomar dos elementos adyacentes en el grafo".$^1$  El grafo está conectado en un círculo ($x_i$ con $x_{i+1}$), y hay un número $M$ de parejas $(i,j)$ conectadas adicionales.  

Para este benchmarking, $N$ varía. Variaremos sobre $M$. $\lambda_{adj}=\lambda_{cross}=2$ dado que la ganancia de tomar una variable es de como mucho 1 punto de energía.


$^1$Sospecho que este problema es equivalente al MAXCUT (el "problema tipo" del QUBO), mas una rotura de la paridad en la simetría $0\leftrightarrow 1$ (favoreciedo un mayor número de 1s). 

## Instalar PyQUBO

In [ ]:
pip install pyqubo

In [ ]:
import numpy as np
import time
import random as rand
import math 

from pyqubo import Array
import neal #alias of SimulatedAnnealingSampler

## Definir funciones de visualización

Me he encontrado con algunos problemas a la hora de gestionar variables mediante PyQUBO.

Para definir tus variables, puedes generar un array, y llama a cada variable de manera secuencial: "x[0]", "x[1]", etc. Esto es mucho más cómodo que ir nombrando tú las variables de una en una, pero provoca que sea difícil ordenar las variables, dado que alfabéticamente "x[11]" está antes que "x[4]" (por ejemplo). Por ello he creado la función "name_to_number", que toma como imput la string "x[$n$]" y devuelve como output el integer $n$.

IMPORTANTE: Existe de manera nativa la opción de asignar a cada variable un número secuencial: model.to_qubo(index_label=True). Sin embargo, esta asignación NO respeta el orden "x[0]"-> 0, "x[1]"->1, etc. sino que asigna las variables en función del peso de sus coeficientes (las variables más importantes aparecerán antes, mediante alguna fórmula que no he llegado a encontrar explícitamente). Esto hace que si has definido tú tus variables con un criterio parecido la asignación automática y la deseada (la que hace la función "name_to_number") sean similares pero NO iguales, lo que puede resultar bastante confuso si no sabes qué está pasando exactamente.


También he definido aquí "draw_graph", una función que genera el grafo de la forma circular que nos interesa.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def name_to_number(var=str):
    #this function reads "x[45]" and returns 45
    return int("".join([cha for cha in var if cha.isdigit()]))
    
def draw_graph(model, show, remove_self_connect=True, font_size=16, figside=16, node_size=60, node_color="c"):
    G = nx.Graph()
    #we read the nodes on integer format
    G.add_nodes_from(sorted([name_to_number(var=var) for var in model.variables])) 
    #we read the edges translating the str elements of model into int elements
    G.add_edges_from({(name_to_number(elem[0]),name_to_number(elem[1])): model.to_qubo(index_label=False)[0][elem] for elem in model.to_qubo(index_label=False)[0]}) #qubo, offset = model.to_qubo()
    if remove_self_connect:
        G.remove_edges_from(nx.selfloop_edges(G))
    
    fig = plt.figure(1, figsize=(figside, figside), dpi=60)
    pos = nx.circular_layout(G)
    if show: 
        nx.draw(G, with_labels=True, pos=pos, node_size=node_size ,font_size=font_size, node_color=node_color)
        plt.show()
    return G

## Discretización de variables

In [ ]:
Vs=10
bVs_per_V=8
x = Array.create('bV', shape=((Vs,bVs_per_V)), vartype='BINARY') #si quieres dos índices, shape=((Vs,bVs_per_V)). Si quieres uno, shape=((Vs*bVs_per_V))
x

Transformación

$$x \Rightarrow \frac{x_0+2x_1+2^2x_2+2^3x_3+...}{10^p} \quad : x\in \mathbb{R} \wedge x_i\in\{0,1\}$$

donde $p\in\mathbb{N}$ es el número de decimales de precisión que requiere $x$. 

El error de la transformación, si el número de variables binarias es lo suficientemente grande para alcanzar el tamaño de $x$, está superiormente acotado por $10^{-p}$. Por ejemplo:

$$30.47=\frac{2^8+2^5+2^4+1}{10}-0.03=30.5-0.03 \Leftarrow 0.03< 10^{-1}$$

## QUBO de ejemplo

In [ ]:
# Establecer los valores de los parámetros
l_adj=2
l_cross=2

In [ ]:
t_ini=time.time()
N=6


n_errors=0

x = Array.create('x', shape=(N), vartype='BINARY')

#crear el hamiltoniano
print("CREATING HAMILTONIAN...")
print("")

H = -sum(x[i] for i in range(N)) #take all possible variables
print("LINEAL TERMS")
for i in range(N):
     print("-x [",i,"]")
print("")


H += l_adj*(   sum((x[i]+x[i+1]-0.5)**2 for i in range(N-1)) + (x[N-1]+x[0]-0.5)**2  )#do not take subsequent variables
print("SUBSEQUENT TERMS")
for i in range(N-1):
    print("( x [",i,"] + x [",i+1,"] - 0.5 )**2")
print("( x [",0,"] + x [",N-1,"] - 0.5 )**2")
print("")


model = H.compile()
bqm = model.to_bqm()


G= draw_graph(model, show=True, figside=5, font_size=40, node_size=2000)
print("")
print("Nodes:", G.nodes())
print("Edges:", G.edges())

#Call ‘to_qubo()’ to get QUBO coefficients.
qubo, offset = model.to_qubo(index_label=False) 
#sin index_label, elem=("('x[2]', 'x[0]') 3.42")
#con index_label, elem=("(2,0) 3.42")


#printear los coeficientes del qubo
print("")
Q=np.zeros((N,N))

for elem in qubo:
    print(elem, qubo[elem])
    i=min(name_to_number(elem[0]),name_to_number(elem[1]))
    j=max(name_to_number(elem[0]),name_to_number(elem[1]))
    Q[i][j]+=qubo[elem]
    
print("Término libre:", offset)
print("")
print(Q)
print("")


#resolver el QUBO
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=10)
decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy) #formato {"x[0]":1, "x[1]":1, "x[2]":0,...}

#formatear la solución para imprimirla
ordered_variables=model.variables.copy()
ordered_variables.reverse() #formato ["x[0]", "x[1]",...]
variable_values=[best_sample.sample[var] for var in ordered_variables] #formato [1,1,0,0,...]
active_variables=[]
for i in range(N):
    if variable_values[i]==1:
        active_variables+=[i] #formato [0,1,3,...]

#printear la solución 

print("N=", N, "n_cross=", 0,"; Active variables of solution:", active_variables, "; Energy of solution:", best_sample.energy, "Término libre:", offset)
print("Cruces prohibidos:", [])
#guardar las características de la solución
for edge in G.edges():
    if (edge[0] in active_variables) and (edge[1] in active_variables):
        if verbose:
            print("¡ERROR! Involucra:", edge)
        n_errors+=1
print("n_errors =", n_errors)
t_fin=time.time()
print("t=", t_fin-t_ini)

color_list=[]
for node in G.nodes():
    if node in active_variables:
        color_list+=["lime"]
    else:
        color_list+=["grey"]
        

draw_graph(model, show=True, figside=5, font_size=40, node_size=2000, node_color=color_list)

In [ ]:
t_ini=time.time()
N=6


n_cross=1
T_max=[(2,5)]


n_errors=0

x = Array.create('x', shape=(N), vartype='BINARY')

#crear el hamiltoniano
print("CREATING HAMILTONIAN...")
print("")

H = -sum(x[i] for i in range(N)) #take all possible variables
print("LINEAL TERMS")
for i in range(N):
     print("-x [",i,"]")
print("")


H += l_adj*(   sum((x[i]+x[i+1]-0.5)**2 for i in range(N-1)) + (x[N-1]+x[0]-0.5)**2  )#do not take subsequent variables
print("SUBSEQUENT TERMS")
for i in range(N-1):
    print("( x [",i,"] + x [",i+1,"] - 0.5 )**2")
print("( x [",0,"] + x [",N-1,"] - 0.5 )**2")
print("")


H += l_cross*(sum((x[i]+x[j]-0.5)**2 for i,j in T_max[:n_cross])  ) #do not take cross-connected variables
print("CROSS TERMS")
for i,j in T_max[:n_cross]:
    print("( x [",i,"] + x [",j,"] - 0.5 )**2")
print("")


model = H.compile()
bqm = model.to_bqm()


G= draw_graph(model, show=True, figside=5, font_size=40, node_size=2000)
print("")
print("Nodes:", G.nodes())
print("Edges:", G.edges())

#Call ‘to_qubo()’ to get QUBO coefficients.
qubo, offset = model.to_qubo(index_label=False) 
#sin index_label, elem=("('x[2]', 'x[0]') 3.42")
#con index_label, elem=("(2,0) 3.42")


#printear los coeficientes del qubo
print("")
Q=np.zeros((N,N))

for elem in qubo:
    print(elem, qubo[elem])
    i=min(name_to_number(elem[0]),name_to_number(elem[1]))
    j=max(name_to_number(elem[0]),name_to_number(elem[1]))
    Q[i][j]+=qubo[elem]
    
print("Término libre:", offset)
print("")
print(Q)
print("")


#resolver el QUBO
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=10)
decoded_samples = model.decode_sampleset(sampleset)
print("")
for sample in decoded_samples: 
    print(list(sample.sample.values()),"-->", -(sample.energy-offset))
print("")
best_sample = min(decoded_samples, key=lambda x: x.energy) #formato {"x[0]":1, "x[1]":1, "x[2]":0,...}

#ordenar la solución
variable_values=[]
for i in range(len(best_sample.sample)):
    key="x["+str(i)+"]"
    variable_values+=[best_sample.sample[key]] #formato [0,1,0,0,1,...]

#formatear la solución para imprimirla
active_variables=[]
for i in range(len(variable_values)):
    if variable_values[i]==1:
        active_variables+=[i] #formato [0,1,3,...]

#printear la solución 

print("N=", N, "n_cross=", n_cross,"; Active variables of solution:", active_variables, "; Energy of solution:", best_sample.energy, "Término libre:", offset," Quality:", -(best_sample.energy-offset))
print("Cruces prohibidos:", T_max)
#guardar las características de la solución
for edge in G.edges():
    if (edge[0] in active_variables) and (edge[1] in active_variables):
        if verbose:
            print("¡ERROR! Involucra:", edge)
        n_errors+=1
print("n_errors =", n_errors)
t_fin=time.time()
print("t=", t_fin-t_ini)

color_list=[]
for node in G.nodes():
    if node in active_variables:
        color_list+=["lime"]
    else:
        color_list+=["grey"]
        

draw_graph(model, show=True, figside=5, font_size=40, node_size=2000, node_color=color_list)

In [ ]:
t_ini=time.time()
N=12

n_cross=3
T_max=[(2,7),(3,7), (4,10)]


n_errors=0

x = Array.create('x', shape=(N), vartype='BINARY')

#crear el hamiltoniano
print("CREATING HAMILTONIAN...")
print("")

H = -sum(x[i] for i in range(N)) #take all possible variables
print("LINEAL TERMS")
for i in range(N):
     print("-x [",i,"]")
print("")


H += l_adj*(   sum((x[i]+x[i+1]-0.5)**2 for i in range(N-1)) + (x[N-1]+x[0]-0.5)**2  )#do not take subsequent variables
print("SUBSEQUENT TERMS")
for i in range(N-1):
    print("( x [",i,"] + x [",i+1,"] - 0.5 )**2")
print("( x [",0,"] + x [",N-1,"] - 0.5 )**2")
print("")


H += l_cross*(sum((x[i]+x[j]-0.5)**2 for i,j in T_max[:n_cross])  ) #do not take cross-connected variables
print("CROSS TERMS")
for i,j in T_max[:n_cross]:
    print("( x [",i,"] + x [",j,"] - 0.5 )**2")
print("")


model = H.compile()
bqm = model.to_bqm()


G= draw_graph(model, show=True, figside=5, font_size=40, node_size=2000)
print("")
print("Nodes:", G.nodes())
print("Edges:", G.edges())

#Call ‘to_qubo()’ to get QUBO coefficients.
qubo, offset = model.to_qubo(index_label=False) 
#sin index_label, elem=("('x[2]', 'x[0]') 3.42")
#con index_label, elem=("(2,0) 3.42")


#printear los coeficientes del qubo
print("")
Q=np.zeros((N,N))

for elem in qubo:
    print(elem, qubo[elem])
    i=min(name_to_number(elem[0]),name_to_number(elem[1]))
    j=max(name_to_number(elem[0]),name_to_number(elem[1]))
    Q[i][j]+=qubo[elem]
    
print("Término libre:", offset)
print("")
print(Q)
print("")


#resolver el QUBO
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=20)
decoded_samples = model.decode_sampleset(sampleset)
very_verbose=True
if very_verbose:
    print("")
    for sample in decoded_samples: 
        print(list(sample.sample.values()),"-->", -(sample.energy-offset))
    print("")
best_sample = min(decoded_samples, key=lambda x: x.energy) #formato {"x[0]":1, "x[4]":1, "x[3]":0,...} DISORDERED!

#ordenar la solución
variable_values=[]
for i in range(len(best_sample.sample)):
    key="x["+str(i)+"]"
    variable_values+=[best_sample.sample[key]] #formato [0,1,0,0,1,...]

#formatear la solución para imprimirla
active_variables=[]
for i in range(len(variable_values)):
    if variable_values[i]==1:
        active_variables+=[i] #formato [0,1,3,...]

#printear la solución 

print("N=", N, "n_cross=", n_cross,"; Active variables of solution:", active_variables, "; Energy of solution:", best_sample.energy, "Término libre:", offset," Quality:", -(best_sample.energy-offset))
print("Cruces prohibidos:", T_max)
#guardar las características de la solución
for edge in G.edges():
    if (edge[0] in active_variables) and (edge[1] in active_variables):
        if verbose:
            print("¡ERROR! Involucra:", edge)
        n_errors+=1
print("n_errors =", n_errors)
t_fin=time.time()
print("t=", t_fin-t_ini)

color_list=[]
for node in G.nodes():
    if node in active_variables:
        color_list+=["lime"]
    else:
        color_list+=["grey"]
        

draw_graph(model, show=True, figside=5, font_size=40, node_size=2000, node_color=color_list)

## Obtener los diccionarios "histo_errors" y "histo_time"

En histo_errors guardaremos, para cada problema, cuántas condiciones ha incumplido su solución. 
En histo_time guardaremos, para cada problema, cuánto tiempo ha tardado en resolverse.

Recordemos que el hamiltoniano a comparar es el siguiente:

$$H=-\sum_i^N x_i + \lambda_{adj}\left[\sum_i^{N-1}\left(x_i + x_{i+1} -\frac{1}{2} \right)^2+ \left(x_N + x_0 -\frac{1}{2}  \right)^2\right]+\lambda_{cross}\left[\sum_{(i, j)\in T}^{M}\left(x_i + x_j -\frac{1}{2}  \right)^2\right]\quad : \forall (i,j) \in T, j \neq \{i-1, \;i\;, i+1\}$$

donde $N$ es el número de variables y $M$ el número de restricciones adicionales que incluimos, además de la restricción de que "para cada pareja adyacente, sólo una cariable puede estar activa".

Para cada $N$, $M$ va desde $0$ hasta $\sqrt N$. 

In [ ]:
very_verbose=False
verbose=False
little_verbose=True
show_graph=True
index_label=True



reset=0
if reset:
    histo_errors={}
    histo_time={}


In [ ]:
global_ini_time=time.time()
N_min=100
N_max=1500

N_jump=100

for N in range(N_min,N_max+1,N_jump): 
    n_cross_min=0
    n_cross_max=math.ceil(0.7*N)
    T_max=[]
    while len(T_max)<n_cross_max:
        i=rand.randint(0,N-1)
        j=rand.randint(0,N-1)
        if abs(i-j)>=2: #if i and j are not the same or adjacent
            if abs(i-j)==N-1: #if i and j are at the extremes
                if very_verbose:
                    print(i,j, "Already connected: they are the first and last elements!")
            else:
                if ((i,j) not in T_max) and ((j,i) not in T_max): #check if the opposite pair is already registered
                    T_max+=[(i,j)]
                elif very_verbose: #elif the pair is registered
                        print(i, j, "Repeated!")
        elif very_verbose: #else, i and j are the same or adjacent
            print(i, j, "Adjacent or equal!")
    if verbose:
        print(T_max)
    
    for n_cross in range(n_cross_min,n_cross_max+1,math.ceil(0.7*N_min)):
        if (N, n_cross) not in histo_errors: #si no se ha caracterizado el problema de ese n_cross...
            t_ini=time.time()
            n_errors=0
            if little_verbose:
                print("------------- N =",N,":   n_cross",n_cross,"/ n_cross_max", n_cross_max,"-------------")
            n_iter=1
            for run in range(n_iter):
                if verbose:
                    print("")
                x = Array.create('x', shape=(N), vartype='BINARY')
    
    
                #crear el hamiltoniano
                
                H = -sum(x[i] for i in range(N)) #take all possible variables
                
                H += l_adj*(   sum((x[i]+x[i+1]-0.5)**2 for i in range(N-1)) + (x[N-1]+x[0]-0.5)**2  )#do not take subsequent variables
                if very_verbose:
                    print("SUBSEQUENTS")
                    for i in range(N-1):
                        print("( x [",i,"] + x [",i+1,"] - 1 )**2")
                    print("( x [",0,"] + x [",N-1,"] - 1 )**2")
                    print("")
                    
                H += l_cross*(sum((x[i]+x[j]-0.5)**2 for i,j in T_max[:n_cross])  ) #do not take cross-connected variables
                
                if very_verbose:
                    print("CROSS")
                    for i,j in T_max[:n_cross]:
                        print("( x [",i,"] + x [",j,"] - 1 )**2")
                    print("")
    
                model = H.compile()
                bqm = model.to_bqm()
    
                
                G= draw_graph(model, show=False)
                if very_verbose:
                    print("Nodes:", G.nodes())
                    print("Edges:", G.edges())
                
                #Call ‘to_qubo()’ to get QUBO coefficients.
                index_label=False
                qubo, offset = model.to_qubo(index_label=index_label) 
                #sin index_label, elem=("('x[2]', 'x[0]') 3.42")
                #con index_label, elem=("(2,0) 3.42")
    
    
                #printear los coeficientes del qubo
                if very_verbose:
                    for elem in qubo:
                        print(elem, qubo[elem])
                    print("Término libre:", offset)
    
                #resolver el QUBO
                sa = neal.SimulatedAnnealingSampler()
                sampleset = sa.sample(bqm, num_reads=1)
                decoded_samples = model.decode_sampleset(sampleset)
                best_sample = min(decoded_samples, key=lambda x: x.energy) #formato {"x[0]":1, "x[1]":1, "x[2]":0,...}

                #ordenar la solución
                variable_values=[]
                for i in range(len(best_sample.sample)):
                    key="x["+str(i)+"]"
                    variable_values+=[best_sample.sample[key]] #formato [0,1,0,0,1,...]
                
                #formatear la solución para imprimirla
                active_variables=[]
                for i in range(len(variable_values)):
                    if variable_values[i]==1:
                        active_variables+=[i] #formato [0,1,3,...]
    
                #printear la solución 
                if verbose:
                    print("N=", N, "n_cross=", n_cross,"; Active variables of solution:", active_variables, "; Energy of solution:", best_sample.energy, "Término libre:", offset)
                    print("Cruces prohibidos:", T_max[:n_cross])
                    
                #guardar las características de la solución
                for edge in G.edges():
                    if (edge[0] in active_variables) and (edge[1] in active_variables):
                        if verbose:
                            print("¡ERROR! Involucra:", edge)
                        n_errors+=1
            histo_errors[(N, n_cross)]=n_errors/n_iter
            t_fin=time.time()
            histo_time[(N, n_cross)]=(t_fin-t_ini)/n_iter
                
        elif verbose:#si sí que se ha caracterizado el problema de ese n_cross...
            print("iteración con N, n_cross =", N,",", n_cross, "ya calculada")

if verbose:
    print("histo_errors",histo_errors)
    print("")
    print("histo_time", histo_time)

#histo_errors_pyqubo=histo_errors.copy()
histo_time_pyqubo=histo_time.copy()

print("Calculations finished. Dictionaries histo_errors and histo_time generated.")
global_fin_time=time.time()

In [ ]:
print("Se han tardado", global_fin_time-global_ini_time,"s en completar el cálculo")

## Graficar los resultados

En estas tres celdas mostramos:

•	El grafo del último problema resuelto (el más grande).

•	Los errores cometidos en función del tamaño del problema. Se puede variar el parámetro "plot_frequency" para que no aparezcan todos los valores de n_cross y facilitar la comparativa.

•	El tiempo requerido para generar la solución en función del tamaño del problema. Se puede variar el parámetro "plot_frequency" para que no aparezcan todos los valores de n_cross y facilitar la comparativa.

In [ ]:
#G= draw_graph(model, show=True, font_size=8)

In [ ]:
verbose=False
very_verbose=False

#histo_errors

In [ ]:
plot_frequency=1#4

#conseguimos los límites del diccionario. Sólo vamos a usar N_fin pero ya que estamos conseguimos N_ini 
N_ini=np.inf
N_fin=0
for elem in histo_errors.keys():
    N=elem[0]
    if N<N_ini:
        N_ini=N
    if N>N_fin:
        N_fin=N


errors_by_n_cross = {n_cross:(N_fin,[]) for n_cross in range(0, math.ceil(0.7*N_fin)+1)} #{n_cross: (N_ini, [errors_of_N_ini, errors_of_N_{ini+1}, errors_of_N_{ini+2},...])
#se inicializa N_ini = N_fin para que si no existe la entrada para ese n_cross itere desde N_ini=N_fin hasta N_fin (esto es, que no haga nada)
#print(errors_by_n_cross)
last_N_added=0

plt.ylabel("Average error per resolution")
plt.xlabel("Variable number N")

checked_n_cross=[]
for elem in histo_errors: #elem=(N, n_cross)
    N=elem[0]
    if N<last_N_added:
        print("ORDER INCONSISTENCY! <-------------------")
    else:
        n_cross=elem[1]
        if len(errors_by_n_cross[n_cross][1])==0:
            if verbose:
                print("n_cross =", n_cross, "ocurre desde N=", N) 
            errors_by_n_cross.update({n_cross: (N, [])}) #get the initial N for the n_cross. we need it to plot the correct range
        avg_errors=histo_errors[elem]
        if very_verbose:
            print("N=",N, "errors_by_n_cross[",n_cross,"][1] =",errors_by_n_cross[n_cross][1])
        errors_by_n_cross.update({n_cross: (errors_by_n_cross[n_cross][0], errors_by_n_cross[n_cross][1]+[avg_errors])})
    last_N_added=N

#plot
plot_cardinal=0
for n_cross in errors_by_n_cross.keys():
    if very_verbose: 
        print("")
        print("n_cross=",n_cross)
    N_ini_del_n_cross= errors_by_n_cross[n_cross][0] #integer
    errors_del_n_cross = errors_by_n_cross[n_cross][1] #list of floats (one for each N between N_ini_del_n_cross and N_fin)
    if very_verbose: 
        print("Range=", range(N_ini_del_n_cross, N_fin+1))
        print("errors_del_n_cross =", errors_del_n_cross) 
        
    if len(errors_del_n_cross)!=0:
        if plot_cardinal%plot_frequency==0:
            '''
            print(n_cross)
            print(range(N_ini_del_n_cross, N_fin+1, N_jump))
            print(errors_del_n_cross)
            '''
            plt.plot(range(N_ini_del_n_cross, N_fin+1, N_jump), errors_del_n_cross, color=(1, 0.75*(n_cross_max-n_cross)/n_cross_max, 0.5+0.5*n_cross/n_cross_max)) #r g b
            last_plotted_ncross=n_cross
            individual_plots=False
            if individual_plots:
                plt.show()
    plot_cardinal+=1
        
    


ncross_ini=np.inf
ncross_fin=0
for elem in histo_errors:
    ncross=elem[1]
    if ncross<ncross_ini:
        ncross_ini=ncross
    if ncross>ncross_fin:
        ncross_fin=ncross

print("Los colores representan el número de condiciones cruzadas incluidas, desde",ncross_ini,"en amarillo, hasta",last_plotted_ncross,"en morado, en saltos de tamaño", plot_frequency,".")



NOTA: vemos que no detectamos ningun error para ninguna de las iteraciones. Todas las entradas de histo_errors son 0.0

In [ ]:
plot_frequency=1 #8

#conseguimos los límites del diccionario. Sólo vamos a usar N_fin pero ya que estamos conseguimos N_ini 
N_ini=np.inf
N_fin=0
for elem in histo_time:
    N=elem[0]
    if N<N_ini:
        N_ini=N
    if N>N_fin:
        N_fin=N

times_by_n_cross = {n_cross:(N_fin,[]) for n_cross in range(0, math.ceil(0.7*N_fin)+1)} #{n_cross: (N_ini, [times_of_N_ini, times_of_N_{ini+1}, times_of_N_{ini+2},...])
last_N_added=0

plt.ylabel("Average time per resolution")
plt.xlabel("Variable number N")

checked_n_cross=[]
for elem in histo_time: #elem=(N, n_cross)
    N=elem[0]
    if N<last_N_added:
        print("ORDER INCONSISTENCY! <-------------------")
    else:
        n_cross=elem[1]
        if len(times_by_n_cross[n_cross][1])==0:
            if verbose:
                print("n_cross =", n_cross, "ocurre desde N=", N) 
            times_by_n_cross.update({n_cross: (N, [])}) #get the initial N for the n_cross. we need it to plot the correct range
        avg_times=histo_time[elem]
        if very_verbose:
            print("N=",N, "times_by_n_cross[n_cross][1] =",times_by_n_cross[n_cross][1])
        times_by_n_cross.update({n_cross: (times_by_n_cross[n_cross][0], times_by_n_cross[n_cross][1]+[avg_times])})
    last_N_added=N

#plot
plot_cardinal=0
for n_cross in times_by_n_cross.keys():
    if very_verbose: 
        print("")
        print("n_cross=",n_cross)
    N_ini_del_n_cross= times_by_n_cross[n_cross][0] #integer
    times_del_n_cross = times_by_n_cross[n_cross][1] #list of floats (one for each N between N_ini_del_n_cross and N_fin)
    if very_verbose: 
        print(range(N_ini_del_n_cross, N_fin+1, N_jump))
        print("")
        print(times_del_n_cross)  
    if len(times_del_n_cross)!=0:
        if plot_cardinal%plot_frequency==0:
            plt.plot(range(N_ini_del_n_cross, N_fin+1, N_jump), times_del_n_cross, color=(1, 0.75*(n_cross_max-n_cross)/n_cross_max, 0.5+0.5*n_cross/n_cross_max)) #r g b
            last_plotted_ncross=n_cross
            individual_plots=False
            if individual_plots:
                plt.show()
    plot_cardinal+=1
    

ncross_ini=np.inf
ncross_fin=0
for elem in histo_time:
    ncross=elem[1]
    if ncross<ncross_ini:
        ncross_ini=ncross
    if ncross>ncross_fin:
        ncross_fin=ncross

print("Los colores representan el número de condiciones cruzadas incluidas, desde",ncross_ini,"en amarillo, hasta",last_plotted_ncross,"en morado, en saltos de tamaño", plot_frequency,".")

# Podemos sacar las siguientes conclusiones:

• PyQUBO puede resolver problemas de al menos 1500 qubits sin cometer errores, muy posiblemente bastante más.

• El tiempo necesario para resolver un problema depende aproximadamente cuadráticamente de la cantidad de variables: desde ~0.05s para N=100 hasta ~5s para N=1500 (en sus variantes de n_cross=0)

• El tiempo necesario para resolver un problema depende aproximadamente linealmente con la conectividad del problema: desde ~5s para 0 conexiones adicionales (1500 conexiones) a ~10s para 1050 conexiones adicionales (2550 conexiones).
